In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from interpretableai import iai
import pandas as pd
from sklearn.metrics import r2_score

1f302ee744f0c5b56a7bdfadaaee583e4266588ca0b8a032e72e0fc0ea74e74d


In [6]:
data = pd.read_csv("features.csv")
data = data.drop(data.columns[0], axis=1)
n_all = data.shape[0]
open_cols = [col for col in data.columns if "Open" in col]
coins = [col.split("_")[1] for col in open_cols]
print(coins)
data = data[65:].reset_index(drop=True)

all_data = "TRUE"
lookback = 100
lookforward = 1

val_size = lookforward * 10
test_size = lookforward * 10

train = data[:-1][:-val_size-test_size]
n_train = len(train)

val = data[:-1][-val_size-test_size:-test_size]
n_val = len(val)

test = data[:-1][-test_size:]
n_test = len(test)

['XRP', 'NEO', 'Stellar', 'Dogecoin', 'Tether', 'XEM', 'TRX', 'NXT', 'Ethereum', 'PPC', 'BNB', 'BTC', 'Litecoin']


In [8]:
randomState = 42

In [9]:
for coin in coins:
    i = 0
    coin_cols = [col for col in data.columns if coin in col]
    df_coin = data[coin_cols]
    x = df_coin.drop(columns="Log_RET_{}".format(coin))[:-1]
    y = df_coin["Log_RET_{}".format(coin)][1:]

    grid = iai.GridSearch(iai.OptimalTreeRegressor(random_seed=randomState, regression_features={'All'}),max_depth=[5],regression_lambda=[0.05])

    nume = []
    deno = []
    y_valid_hat_coin = np.array([])
    # Validation
    if all_data == "TRUE":
        for i in range(0, n_val, lookforward):
            x_train = x.iloc[0:n_train + i]
            y_train = y.iloc[0:n_train + i]
            x_valid = x.iloc[n_train + i :n_train + i + lookforward]
            y_valid = y.iloc[n_train + i :n_train + i + lookforward]
            grid.fit(x_train, y_train)

            #get tree
            grid.write_html("results/" + coin + "_tree.html")
            bestLearner = grid.get_learner()
            #tree_learners.append(bestLearner)

            y_valid_hat_wind = bestLearner.predict(x_valid)
            y_valid_hat_coin = np.append(y_valid_hat_coin, y_valid_hat_wind, axis=0)
            nume.append((y_valid_hat_wind-y_valid.values)**2)
            deno.append((y_valid.values-y_train.mean())**2)

    val["Log_RET_{}_pred".format(coin)] = y_valid_hat_coin

    r2 = r2_score(val["Log_RET_{}".format(coin)][:-1], val["Log_RET_{}_pred".format(coin)][1:])
    print("Validation R2 for {} is {}".format(coin, r2))

    nume = np.array(nume)
    nume = nume.reshape(nume.shape[0]*nume.shape[1],)
    deno = np.array(deno)
    deno = deno.reshape(deno.shape[0]*deno.shape[1],)
    osr2 = 1-sum(nume)/sum(deno)
    print("Validation OSR2 for {} is {}".format(coin, osr2))

    mse = np.mean((val["Log_RET_{}".format(coin)][:-1] - val["Log_RET_{}_pred".format(coin)][1:])**2)
    print("Validation MSE for {} is {}".format(coin, mse))

    """
    # Test
    y_test_hat_coin = np.array([])
    if all_data == "TRUE":
        for i in range(0, n_test, lookforward):
            x_train = x.iloc[0:n_train + n_val + i]
            y_train = y.iloc[0:n_train + n_val + i]
            x_test = x.iloc[n_train + n_val + i :n_train + n_val + i + lookforward]
            y_test = y.iloc[n_train + n_val + i :n_train + n_val + i + lookforward]
            grid.fit(x_train, y_train)
            bestLearner = grid.get_learner()
            y_test_hat_wind = bestLearner.predict(x_test)
            y_test_hat_coin = np.append(y_test_hat_coin, y_test_hat_wind, axis=0)

    test["Log_RET_{}_pred".format(coin)] = y_test_hat_coin
    """

# Save predictions
val.to_csv("results/optTree_val.csv")
test.to_csv("results/optTree_val_test.csv")

[ Warning: This copy of Interpretable AI software is for academic purposes only and not for commercial use.
┌ Warning: Interpretable AI license expires soon: 2022-12-31T00:00:00. If you need to renew, please send us the following machine ID:
└ 1f302ee744f0c5b56a7bdfadaaee583e4266588ca0b8a032e72e0fc0ea74e74d
┌ Warning: Numeric instability detected: objective value after iteration did not match the value that was expected so this change was rejected.
│ 
│ Possible causes include:
│ - using linear regression in the leaves with a small `minbucket`
│ - using `:tweedie` criterion with a large number of samples
│ 
└ If none of these apply, please let us know so we can investigate.
/var/folders/j2/7j0fctvs2258gcs69mmjmlsr0000gn/T/ipykernel_1860/1292975939.py:34: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `ser

Validation R2 for XRP is 0.2866861375032508
Validation OSR2 for XRP is -0.09803996323582598
Validation MSE for XRP is 0.0002889953499743604


/var/folders/j2/7j0fctvs2258gcs69mmjmlsr0000gn/T/ipykernel_1860/1292975939.py:34: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  r2 = r2_score(val["Log_RET_{}".format(coin)][:-1], val["Log_RET_{}_pred".format(coin)][1:])
/var/folders/j2/7j0fctvs2258gcs69mmjmlsr0000gn/T/ipykernel_1860/1292975939.py:44: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  mse = np.mean((val["Log_RET_{}".format(coin)][:-1] - val["Log_RET_{}_pred".format(coin)][1:])**2)
